# Reframing Time Series Forecasting
You do not have to model your time series forecast problem as-is. There are many ways to reframe your forecast problem that can both simplify the prediction problem and potentially expose more or di↵erent information to be modeled. A reframing can ultimately result in better and/or more robust forecasts.

## Benefit of Reframing
Reframing your problem is the idea of exploring alternate perspectives on what is to be predicted. There are two potential benefits to exploring alternate framings of your time series forecast problem:
1. Simplify your problem.
2. Provide the basis for an ensemble forecast.
Both benefits ultimately have the result of leading to more skillful and/or more robust forecasts.

### Simplify your problem
Perhaps the largest wins on a forecasting project can come from a reframing of the problem. This is because the structure and type of prediction problem has so much more impact than the choice of data transforms, choice of model, or the choice of model hyperparameters. It is the biggest lever in a project and must be carefully considered.

### Enesemble Forecast
In addition to changing the problem you are working on, reframing plays another role: it can provide you with a suite of different, but highly related problems that you can model. The benefit of this is that the framings may be different enough to require differences in data preparation and modeling methods.

Models of differing perspectives on the same problem may capture different information from the input and in turn result in predictions that are skillful, but in different ways. These predictions may be combined in an ensemble to result in a more skillful or more robust forecast.

## Regression Framings

Most time series prediction problems are regression problems, requiring the prediction of a real-valued output. Below are 5 different ways that this prediction problem could be re-phrased as an alternate regression problem:

1. Forecast the change in the minimum temperature compared to the previous day. 
2. Forecast the minimum temperature relative to the average from the past 14 days. 
3. Forecast the minimum temperature relative to the average the same month last year.
4. Forecast the minimum temperature rounded to the nearest 5 degrees Celsius.
5. Forecast the average minimum temperature for the next 7 days.

Making the temperature relative is a linear transform and may not make the problem simpler and easier to predict, but it may shake loose new ideas or even new sources of data that you may consider. It also may help you think more clearly about how exactly the forecast will be used and what the hard requirements on that forecasted value actually are.

Transforming the granularity of a prediction problem does change the difficulty of the problem and can be very useful if the requirements of the problem permit such redefinitions.

In [5]:
# Reframe precision of regression forecast; predict daily temperature rounded to nearest 5 degress
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from math import floor

#load data
series = read_csv('daily-minimum-temperatures.csv', header=0, parse_dates=[0], 
                  index_col = 0, nrows=3650, squeeze=True)
series = series.map(lambda x: x.lstrip('?'))
series = series.astype(float)

#Create Lagged Set
values = DataFrame(series.values)
dataframe = concat([values.shift(1), values], axis=1)
dataframe.columns = ['t','t+1']

#round forecast to nearest 5
for i in range(len(dataframe['t+1'])):
    dataframe['t+1'][i] = int(dataframe['t+1'][i] / 5) * 5.0
print(dataframe.head(5))

      t   t+1
0   NaN  20.0
1  20.7  15.0
2  17.9  15.0
3  18.8  10.0
4  14.6  15.0


The problem is defined as given the minimum temperature the day before in degrees Celsius, the minimum to the nearest 5 degrees.

## Classification Framings

Classification involves predicting categorical or label outputs (like hot and cold). Below are 5 di↵erent ways that this prediction problem can be rephrased as a classification problem:

1. Forecast whether a minimum temperature will be cold, moderate, or warm. 
2. Forecast whether a change in minimum temperature will be small or large.
3. Forecast whether the minimum temperature will be a monthly minimum or not.
4. Forecast whether the minimum will be higher or lower than the minimum in the previous year.
5. Forecast whether the minimum temperature will rise or fall over the next 7 days.

A move to classification can simplify the prediction problem. This approach opens up ideas on labels as well as binary classification framings.

The native regression representation of the output variable means that most classification framings are likely to keep the ordinal structure (e.g. cold, moderate, hot). Meaning that there is an ordered relationship between the classes being predicted, which may not be the case when predicting labels like dog and cat.

The ordinal relationship permits both a hard classification problem as well as an integer prediction problem that can be post-hoc rounded into a specific category. Below is an example of transforming the Minimum Daily Temperatures forecast problem to a classification problem where each temperature value is an ordinal value of cold, moderate, or hot. These labels are mapped to integer values, defined as:

- 0 (cold): < 10 degrees Celsius.
- 1 (moderate):   10 and < 25 degrees Celsius. 
- 2 (hot):   25 degrees Celsius.

In [6]:
# reframe regression as classification
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from math import floor

#load data
series = read_csv('daily-minimum-temperatures.csv', header=0, parse_dates=[0], 
                  index_col = 0, nrows=3650, squeeze=True)
series = series.map(lambda x: x.lstrip('?'))
series = series.astype(float)

#Create Lagged Set
values = DataFrame(series.values)
dataframe = concat([values.shift(1), values], axis = 1)
dataframe.columns = ['t', 't+1']

#make discrete
for i in range(len(dataframe['t+1'])):
    value = dataframe['t+1'][i]
    if value < 10.0:
        dataframe['t+1'][i] = 0
    elif value >= 25.0:
        dataframe['t+1'][i] = 2
    else:
        dataframe['t+1'][i] = 1
print(dataframe.head(5))

      t  t+1
0   NaN  1.0
1  20.7  1.0
2  17.9  1.0
3  18.8  1.0
4  14.6  1.0


## Time horizon Framings

Another axis that can be varied is the time horizon. The time horizon is the number of time steps in the future that are being predicted. Below are 5 different ways that this prediction problem can be re-phrased as a different time horizon:
1. Forecast the minimum temperature for the next 7 days.
2. Forecast the minimum temperature in 30 days time.
3. Forecast the average minimum temperature next month.
4. Forecast the day in the next week that will have the lowest minimum temperature. 
5. Forecast one year of minimum temperature values.

You may be able to forecast far into the future, but the skill is likely going to vary, degrading further into the future you project. When thinking through the horizon of the forecast, also consider the minimum acceptable performance of forecasts.

In [7]:
# reframe regression as classification
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from math import floor

#load data
series = read_csv('daily-minimum-temperatures.csv', header=0, parse_dates=[0], 
                  index_col = 0, nrows=3650, squeeze=True)
series = series.map(lambda x: x.lstrip('?'))
series = series.astype(float)

#Create Lagged Set
values = DataFrame(series.values)
dataframe = concat([values.shift(1), values, values.shift(-1), values.shift(-2), values.shift(-3), 
                   values.shift(-4), values.shift(-5), values.shift(-6)], axis =1)
dataframe.columns = ['t', 't+1', 't+2', 't+3', 't+4', 't+5', 't+6', 't+7']
print(dataframe.head(14))

       t   t+1   t+2   t+3   t+4   t+5   t+6   t+7
0    NaN  20.7  17.9  18.8  14.6  15.8  15.8  15.8
1   20.7  17.9  18.8  14.6  15.8  15.8  15.8  17.4
2   17.9  18.8  14.6  15.8  15.8  15.8  17.4  21.8
3   18.8  14.6  15.8  15.8  15.8  17.4  21.8  20.0
4   14.6  15.8  15.8  15.8  17.4  21.8  20.0  16.2
5   15.8  15.8  15.8  17.4  21.8  20.0  16.2  13.3
6   15.8  15.8  17.4  21.8  20.0  16.2  13.3  16.7
7   15.8  17.4  21.8  20.0  16.2  13.3  16.7  21.5
8   17.4  21.8  20.0  16.2  13.3  16.7  21.5  25.0
9   21.8  20.0  16.2  13.3  16.7  21.5  25.0  20.7
10  20.0  16.2  13.3  16.7  21.5  25.0  20.7  20.6
11  16.2  13.3  16.7  21.5  25.0  20.7  20.6  24.8
12  13.3  16.7  21.5  25.0  20.7  20.6  24.8  17.7
13  16.7  21.5  25.0  20.7  20.6  24.8  17.7  15.5
